In [1]:
# Import Library
# text preprocessing
import pandas as pd
import numpy as np
import csv
import requests
import io
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re # regular expression
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # stemming indonesian language
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.corpus import stopwords
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer # to create Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer # tfid Vector 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score # confussion matrix
from sklearn.preprocessing import LabelEncoder # to convert classes to number 
from sklearn.model_selection import train_test_split  # for splitting data 
from sklearn.metrics import accuracy_score # to calculate accuracy
from sklearn.pipeline import Pipeline
#from mlxtend.plotting import plot_confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset = pd.read_csv(r'before_preprocessing.csv')

In [3]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1


In [4]:
dataset.isnull().sum()

review_id      0
review_text    0
price          0
packaging      0
product        1
aroma          0
dtype: int64

In [5]:
dataset = dataset.fillna(method="ffill")
dataset.isnull().sum()

review_id      0
review_text    0
price          0
packaging      0
product        0
aroma          0
dtype: int64

In [6]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1


In [7]:
# casefolding 

def clean(dataset):
  temp_text = []

  for txt in dataset:
    # removal of @name[mention]
    txt = re.sub(r"(?:\@|https?\://)\S+", "", txt)

    # removal of links[https://blabala.com]
    # tw = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", tw)
    txt = re.sub(r"http\S+", "", txt)

    # removal of new line
    txt = re.sub('\n', '', txt)

    # removal of RT
    txt = re.sub('RT', '', txt)

    # removal of punctuations and numbers
    txt = re.sub("[^a-zA-Z^']", " ", txt)
    txt = re.sub(" {2,}", " ", txt)

    # remove leading and trailing whitespace
    txt = txt.strip()

    # remove whitespace with a single space
    txt = re.sub(r'\s+', ' ', txt)

    # convert text to Lowercase
    text = txt.lower();
    temp_text.append(txt)
  return temp_text 

dataset['Clean_text'] = clean(dataset['review_text'])
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,sunscreen termahal yang pernah gue beli ini ka...
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,ini enak banget dipakainya enteng banget diwaj...
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,the description is quite right produk ini eman...
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,bisa untuk wajah dan badan dengan harga yang s...
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,saya beli produk ini karena suka banget wangin...


In [8]:
#TOKENISASI
def token(dataset):
  return dataset.apply(nltk.word_tokenize)

dataset['Clean_text'] = token(dataset['Clean_text'])
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,"[sunscreen, termahal, yang, pernah, gue, beli,..."
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,"[ini, enak, banget, dipakainya, enteng, banget..."
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,"[the, description, is, quite, right, produk, i..."
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,"[bisa, untuk, wajah, dan, badan, dengan, harga..."
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,"[saya, beli, produk, ini, karena, suka, banget..."


In [9]:
dataset['Clean_text'] = dataset['Clean_text'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,sunscreen termahal yang pernah gue beli ini ka...
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,ini enak banget dipakainya enteng banget diwaj...
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,the description is quite right produk ini eman...
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,bisa untuk wajah dan badan dengan harga yang s...
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,saya beli produk ini karena suka banget wangin...


In [10]:
slang_word = requests.get('https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt').text
dict_slang = eval(slang_word)

df_acronym = pd.read_csv('https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/acronym.txt', header=None, sep='=')
df_acronym.columns = ['Singkatan', 'kepanjangan']
df_acronym.kepanjangan = df_acronym.kepanjangan.apply(lambda x: x.strip().lower())
df_acronym.Singkatan = df_acronym.Singkatan.apply(lambda x: x.strip().lower())
dict_singkatan = pd.Series(df_acronym.kepanjangan.values,index=df_acronym.Singkatan).to_dict()

dict_clean = {**dict_singkatan, **dict_slang}

In [11]:
def preprocessing_slang_singkatan(review_text, dict_clean=dict_clean):
  regex = r"\b(?:"+"|".join(re.escape(word) for word in dict_clean) + r")\b"
  reobj = re.compile(regex, re.I)
  
  return reobj.sub(lambda x:dict_clean[x.group(0)], review_text)

In [12]:
def clean_all(x, dict_clean=dict_clean):
  x = preprocessing_slang_singkatan(x, dict_clean=dict_clean)
  return x

In [13]:
tqdm.pandas()

In [14]:
dataset['Clean_text'] = dataset.Clean_text.progress_apply(lambda x: clean_all(x, dict_clean=dict_clean))

100%|██████████| 3960/3960 [00:30<00:00, 131.88it/s]


In [16]:
dataset.to_csv('data_preprocessing_tanpastopword_tanpastemming.csv', index=False, sep= ';')

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer
tfidf_transformer = TfidfTransformer()
tfidf_vectorizer = TfidfVectorizer()
count_vectorize = CountVectorizer()

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'Pusing satu kali',
    'Pusing dua kali',
    'Pusing tiga kali',
    'Pusing lagi']

vectorizer = TfidfVectorizer()

# contoh
a = vectorizer.fit_transform(corpus)
a.toarray()

array([[0.        , 0.49248889, 0.        , 0.40264194, 0.77157901,
        0.        ],
       [0.77157901, 0.49248889, 0.        , 0.40264194, 0.        ,
        0.        ],
       [0.        , 0.49248889, 0.        , 0.40264194, 0.        ,
        0.77157901],
       [0.        , 0.        , 0.88654763, 0.46263733, 0.        ,
        0.        ]])

In [19]:
a = dataset['Clean_text']
# Count TF_IDF Vectorizer
count_vectorizer = CountVectorizer()
count_vector = count_vectorizer.fit_transform(dataset['Clean_text'])
count_vector.shape

(3960, 10963)

In [20]:
a = dataset['Clean_text']
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer = tfidf_vectorizer.fit_transform(a)
print(tfidf_vectorizer)


  (0, 7417)	0.04546082596079838
  (0, 5801)	0.10022107864701454
  (0, 4215)	0.0497239334950299
  (0, 946)	0.17478849883268197
  (0, 8096)	0.06430206170959138
  (0, 7998)	0.17592609474249327
  (0, 1722)	0.19973861733531917
  (0, 1994)	0.0363582289446547
  (0, 2775)	0.15640735531267105
  (0, 9959)	0.1381351394396345
  (0, 10611)	0.14524491086503974
  (0, 1026)	0.08851750056979785
  (0, 6660)	0.06627142373460529
  (0, 5966)	0.21874527692521886
  (0, 9357)	0.10262102089780652
  (0, 10185)	0.11032370654189423
  (0, 4152)	0.2128484357635605
  (0, 1240)	0.23742429129686818
  (0, 7770)	0.21731957279211928
  (0, 1552)	0.14606200652283324
  (0, 10773)	0.13430150011000042
  (0, 6261)	0.15305690659934088
  (0, 10220)	0.12313365965772284
  (0, 9505)	0.1121956449073298
  (0, 3849)	0.18239836846897378
  :	:
  (3959, 10515)	0.06880839253835738
  (3959, 9913)	0.06889003465547967
  (3959, 4146)	0.06868645843912446
  (3959, 6266)	0.127276554905328
  (3959, 9142)	0.07429063528889157
  (3959, 9181)	0.11180

In [21]:
tfidf_vectorizer.shape

(3960, 10963)

In [22]:
vectorizer = TfidfVectorizer(max_features=1000)
vectors = vectorizer.fit_transform(dataset.Clean_text)
words_dataset = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_dataset.head()

,aceh,acid,acne,ada,adalah,adat,adem,administrasi,advanced,affordable,...,would,wow,yaa,yah,yaitu,yang,yes,you,your,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.053964,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.058069,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.138491,...,0.0,0.0,0.0,0.0,0.0,0.043132,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.096484,0.0,0.0,0.0,0.0


In [23]:
#seleksi fitur
import numpy as np
from scipy import stats
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import ClassifierChain
from sklearn import datasets
from sklearn.feature_selection import SelectKBest,chi2 , f_regression, SelectFromModel, SelectPercentile
from sklearn.preprocessing import LabelBinarizer
from sklearn.datasets import load_iris
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MultiLabelBinarizer

In [24]:
x = tfidf_vectorizer.copy()

In [25]:
y = np.array(dataset[['price','packaging','product','aroma']])

In [26]:
x.shape, y.shape

((3960, 10963), (3960, 4))

In [27]:
Y = MultiLabelBinarizer().fit_transform(y)

In [34]:
x_new = SelectKBest(chi2, k=857).fit_transform(x, Y)

In [35]:
print(x_new)

  (0, 439)	0.10022107864701454
  (0, 788)	0.11032370654189423
  (0, 106)	0.23742429129686818
  (0, 792)	0.12313365965772284
  (0, 736)	0.1121956449073298
  (0, 169)	0.14524491086503974
  (0, 720)	0.07224518263333046
  (0, 849)	0.11931974331632357
  (0, 817)	0.0961672866271731
  (0, 339)	0.11787321199888479
  (0, 68)	0.07447483468747404
  (0, 665)	0.1437151650176708
  (0, 603)	0.09946328576531863
  (0, 783)	0.19721485428737037
  (0, 756)	0.22362293480201823
  (1, 672)	0.15350429376134214
  (1, 192)	0.12586535255833364
  (1, 310)	0.1101064759247273
  (1, 384)	0.12586535255833364
  (1, 370)	0.16270963405818173
  (1, 12)	0.16270963405818173
  (1, 426)	0.16270963405818173
  (1, 536)	0.08423050023730182
  (1, 143)	0.11682916592385413
  (1, 329)	0.15015634635066596
  :	:
  (3958, 614)	0.08521944206378912
  (3958, 707)	0.16477898385145134
  (3958, 675)	0.17678866088244516
  (3958, 688)	0.11598608804067734
  (3958, 692)	0.08204936294451631
  (3958, 446)	0.10597090529957434
  (3958, 439)	0.09010

In [36]:
from sklearn.model_selection import train_test_split
x_new_train, x_new_test, Y_train, Y_test = train_test_split(x_new,Y, test_size=0.2, random_state=0)

In [37]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor


knn = KNeighborsClassifier(n_neighbors=11)
## Fit the model on the training data.
classifier = MultiOutputClassifier(knn, n_jobs=-1)
classifier.fit(x_new_train, Y_train)
## See how the model performs on the test data.
predictions = classifier.predict(x_new_test)

In [38]:
knn_skor=classifier.score(x_new_test,np.array(Y_test))
print(knn_skor * 100 ,'%')

58.838383838383834 %


In [39]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import recall_score
from sklearn.metrics import  multilabel_confusion_matrix

multilabel = multilabel_confusion_matrix(Y_test,predictions)
f1_score = metrics.f1_score(Y_test,predictions, average='micro', labels=np.unique(predictions))
precision_score = metrics.precision_score(Y_test,predictions, average='micro', labels=np.unique(predictions))
recall_score = metrics.recall_score(Y_test,predictions, average='micro', labels=np.unique(predictions))
        
print('akurasi f1 score        :', f1_score* 100 ,'%')
print('akurasi precision score :', precision_score* 100 ,'%')
print('akurasi recall score    :', recall_score* 100 ,'%')

akurasi f1 score        : 83.2983193277311 %
akurasi precision score : 98.50931677018635 %
akurasi recall score    : 72.1565059144677 %
